In [532]:
%%html
<h1> Set up Jira Client</h1>

In [508]:
from jira import JIRA
import numpy as np
import pandas as pd
import xlsxwriter

import json

with open('jira.json') as json_data_file:
    data = json.load(json_data_file)
    username = data['auth']['username']
    password = data['auth']['password']
    bugqueryadd = data['bugqueryadd']
    epicqueryadd = data['epicqueryadd']
    storyqueryadd = data['storyqueryadd']
    domain = data['domain']
    columns = data['columns']
    fields = data['fields']
    outfile = data['outfile']

if not domain:
    domain = raw_input("Jira Domain (e.g https://XXX:PPP/jira): ")

if not username:
    username = raw_input("Username: ")

if not password:
    password = getpass.getpass("Password: ")
   
def get_jira_client(domain, username, password):
    options = {'server': domain}
    return JIRA(options, basic_auth=(username, password))

def print_jira_issue(issue):
    print (issue['key'], ":", issue['fields']['summary'])
    
writer = pd.ExcelWriter(outfile)
jira = get_jira_client(domain, username, password)

In [410]:
%%html
<h1> Load Stories, Epics and Bugs</h1>

In [411]:
epics = jira.search_issues('type=epic and ' + epicqueryadd, json_result=True, maxResults=1000, fields = fields)

In [412]:
stories = jira.search_issues('type=story and ' + storyqueryadd, json_result=True, maxResults=1000, fields = fields, expand='changelog')

In [509]:
bugs = jira.search_issues('type=bug and ' + bugqueryadd, json_result=True, maxResults=1000, fields = fields)

In [414]:
%%html
<h1> Set up the Dataframes for Stories and Epics</h1>

In [415]:
#prep the stories and epics dataframes
#fix the column names
#extract comment data 
#extract all the history from stories and build all the workflow fields

for issue in stories['issues']:
    #merge the textual fields of comments, summary
    alltext = [comment['body'] for comment in issue['fields']['comment']['comments']]
    if (issue['fields']['summary'] != None):
        alltext.append(issue['fields']['summary'])
    if (issue['fields']['description'] != None):
        alltext.append(issue['fields']['description'])
    try:
        issue['fields']['textinfo'] = ' '.join(alltext)
    except TypeError:
        print(alltext)

    #for stories only, record the important parts of change log as separate columns
    
    issue['fields']['Open Set By'] = []
    issue['fields']['Approval Set By'] = []
    issue['fields']['Closed Set By'] = []
    issue['fields']['Code Review Set By'] = []
    issue['fields']['In Analysis Set By'] = []
    issue['fields']['In Progress Set By'] = []
    issue['fields']['In UI/UX Set By'] = []
    issue['fields']['Ready for Estimation Set By'] = []
    issue['fields']['Testing Set By'] = []
    
    changelog = issue['changelog']
    for history in changelog['histories']:
        for item in history['items']:
            if item['field'] == 'status':
                #need to ensure if there are multiple times a certain status is updated, we capture it
                #the first or last time based on the specific status.
                timestamp = pd.to_datetime(history['created'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
                event = item['toString'] + ' ' + 'Set By'
                author = history['author']['name']
                issue['fields'][event].append((author, timestamp))
                #issue['fields'][item['toString'] + ' ' + 'Set To Date'] = history['created']
                #issue['fields'][item['toString'] + ' ' + 'Set By'] = history['author']['name']
                
    issue['fields']['Open Set By'] = min(issue['fields']['Open Set By'], key = lambda t: t[1]) if issue['fields']['Open Set By'] else None
    issue['fields']['Approval Set By'] = max(issue['fields']['Approval Set By'], key = lambda t: t[1]) if issue['fields']['Approval Set By'] else None
    issue['fields']['Closed Set By'] = max(issue['fields']['Closed Set By'], key = lambda t: t[1]) if issue['fields']['Closed Set By'] else None
    issue['fields']['Code Review Set By'] = min(issue['fields']['Code Review Set By'], key = lambda t: t[1]) if issue['fields']['Code Review Set By'] else None
    issue['fields']['In Analysis Set By'] = min(issue['fields']['In Analysis Set By'], key = lambda t: t[1]) if issue['fields']['In Analysis Set By'] else None
    issue['fields']['In Progress Set By'] = min(issue['fields']['In Progress Set By'], key = lambda t: t[1]) if issue['fields']['In Progress Set By'] else None
    issue['fields']['In UI/UX Set By'] = min(issue['fields']['In UI/UX Set By'], key = lambda t: t[1]) if issue['fields']['In UI/UX Set By'] else None
    issue['fields']['Ready for Estimation Set By'] = min(issue['fields']['Ready for Estimation Set By'], key = lambda t: t[1]) if issue['fields']['Ready for Estimation Set By'] else None
    issue['fields']['Testing Set By'] = min(issue['fields']['Testing Set By'], key = lambda t: t[1]) if issue['fields']['Testing Set By'] else None

    issue['fields']['Open Set To Date'] = issue['fields']['Open Set By'][1] if issue['fields']['Open Set By'] else None
    issue['fields']['Open Set By'] = issue['fields']['Open Set By'][0] if issue['fields']['Open Set By'] else None
    
    issue['fields']['Approval Set To Date'] = issue['fields']['Approval Set By'][1] if issue['fields']['Approval Set By'] else None
    issue['fields']['Approval Set By'] = issue['fields']['Approval Set By'][0] if issue['fields']['Approval Set By'] else None
    
    issue['fields']['Closed Set To Date'] = issue['fields']['Closed Set By'][1] if issue['fields']['Closed Set By'] else None
    issue['fields']['Closed Set By'] = issue['fields']['Closed Set By'][0] if issue['fields']['Closed Set By'] else None
    
    issue['fields']['Code Review Set To Date'] = issue['fields']['Code Review Set By'][1] if issue['fields']['Code Review Set By'] else None
    issue['fields']['Code Review Set By'] = issue['fields']['Code Review Set By'][0] if issue['fields']['Code Review Set By'] else None
    
    issue['fields']['In Analysis Set To Date'] = issue['fields']['In Analysis Set By'][1] if issue['fields']['In Analysis Set By'] else None
    issue['fields']['In Analysis Set By'] = issue['fields']['In Analysis Set By'][0] if issue['fields']['In Analysis Set By'] else None
    
    issue['fields']['In Progress Set To Date'] = issue['fields']['In Progress Set By'][1] if issue['fields']['In Progress Set By'] else None
    issue['fields']['In Progress Set By'] = issue['fields']['In Progress Set By'][0] if issue['fields']['In Progress Set By'] else None
    
    issue['fields']['In UI/UX Set To Date'] = issue['fields']['In UI/UX Set By'][1] if issue['fields']['In UI/UX Set By'] else None
    issue['fields']['In UI/UX Set By'] = issue['fields']['In UI/UX Set By'][0] if issue['fields']['In UI/UX Set By'] else None
    
    issue['fields']['Ready for Estimation Set To Date'] = issue['fields']['Ready for Estimation Set By'][1] if issue['fields']['Ready for Estimation Set By'] else None
    issue['fields']['Ready for Estimation Set By'] = issue['fields']['Ready for Estimation Set By'][0] if issue['fields']['Ready for Estimation Set By'] else None
    
    issue['fields']['Testing Set To Date'] = issue['fields']['Testing Set By'][1] if issue['fields']['Testing Set By'] else None
    issue['fields']['Testing Set By'] = issue['fields']['Testing Set By'][0] if issue['fields']['Testing Set By'] else None
    
    
for issue in epics['issues']:
    alltext = [comment['body'] for comment in issue['fields']['comment']['comments']]
    alltext.append(issue['fields']['summary'])
    #alltext.append(issue['fields']['description'])
    issue['fields']['textinfo'] = ' '.join(alltext)

epic_list = []
for epic in epics['issues']:
    epic['fields']['key'] = epic['key']
    epic_list.append(epic['fields'])

epics_df = pd.DataFrame(epic_list)

story_list = []
for story in stories['issues']:
    story['fields']['key'] = story['key']
    story_list.append(story['fields'])

stories_df = pd.DataFrame(story_list)

#replacement of custom field's by their names is only done inside the dataframe
# Fetch all fields
allfields=jira.fields()
# Make a map from field name -> field id
nameMap = {field['name']:field['id'] for field in allfields}
idMap = {field['id']:field['name'] for field in allfields}

for column in epics_df.columns:
    if ('custom' in column):
        epics_df.rename(columns={column: idMap[column]}, inplace=True)

for column in stories_df.columns:
    if ('custom' in column):
        stories_df.rename(columns={column: idMap[column]}, inplace=True)

stories_df['Team'] = stories_df['Team'].apply(lambda x: x[0].get('value') if (type(x) == list) else None)
stories_df['status'] = stories_df['status'].apply(lambda x: x.get('name'))
stories_df['reporter'] = stories_df['reporter'].apply(lambda x: x.get('name'))
stories_df['fixVersions'] = stories_df['fixVersions'].apply(lambda x: x[0]['name'] if ((type(x) == list) and x and (type(x[0]) == dict)) else None)
stories_df['Platform'] = stories_df['Platform'].apply(lambda x: x[0].get('value'))

#insert a column for jira link
stories_df['story_link'] = '=HYPERLINK("' + domain + '/browse/' + stories_df['key'] + '","' + stories_df['key'] + '")'

#Change the string time fields into the python datetime structures

from datetime import datetime
from datetime import timedelta

#stories_df['Approval Set To Date'] = pd.to_datetime(stories_df['Approval Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['Closed Set To Date'] = pd.to_datetime(stories_df['Closed Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['Code Review Set To Date'] = pd.to_datetime(stories_df['Code Review Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['In Analysis Set To Date'] = pd.to_datetime(stories_df['In Analysis Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['In Progress Set To Date'] = pd.to_datetime(stories_df['In Progress Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['In UI/UX Set To Date'] = pd.to_datetime(stories_df['In UI/UX Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['Open Set To Date'] = pd.to_datetime(stories_df['Open Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['Ready for Estimation Set To Date'] = pd.to_datetime(stories_df['Ready for Estimation Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
#stories_df['Testing Set To Date'] = pd.to_datetime(stories_df['Testing Set To Date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')

In [417]:
%%html
<h1> Set up the Dataframes for Sprints</h1>

In [418]:
#extract the sprint information from the sprints field and create a separate sprints-issue dataframe
#this is only possible once we have the stories dataframe

from functools import reduce

#Takes a list of sprints of the form:
#['com.atlassian.greenhopper.service.sprint.Sprint@1b7eb58a[id=519,rapidViewId=219,state=CLOSED,name=Knight Riders Sprint 2018 - 22,startDate=2018-05-23T21:16:06.149+05:30,endDate=2018-06-05T19:44:00.000+05:30,completeDate=2018-06-06T20:45:27.547+05:30,sequence=519]',
# 'com.atlassian.greenhopper.service.sprint.Sprint@2a28663d[id=542,rapidViewId=219,state=ACTIVE,name=Knight Riders Sprint 2018-23,startDate=2018-06-06T22:14:10.412+05:30,endDate=2018-06-19T20:42:00.000+05:30,completeDate=<null>,sequence=542]']
# and returns one list with a dictionary object for each sprint located. The object also contains the issue key
# the other is 
# we return a dictionary
def getSprintInfo(issueKey, sprint):
    #locate the part in square braces
    start = sprint.find('[') + 1
    end = sprint.find(']', start)
    dict_sprint = dict(x.split('=') for x in sprint[start:end].split(','))
    dict_sprint['issue_key'] = issueKey
    return dict_sprint

#we return a list of dictionaries, where each dictionary is a sprint paired with the issue.
def getSprints (issueKey, sprints):
    if type(sprints) == list:
        return [getSprintInfo(issueKey, sprint) for sprint in sprints]
    else:
        return []

x1 = []
for index, row in stories_df.iterrows():
    x1 = x1 + (getSprints(row['key'], row['Sprint']))

sprints_df =  pd.DataFrame(x1)
sprints_df['endDate'] = pd.to_datetime(sprints_df['endDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
sprints_df['startDate'] = pd.to_datetime(sprints_df['startDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
sprints_df['completeDate'] = pd.to_datetime(sprints_df['completeDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')

In [419]:
%%html
<h1> Set up the Dataframes for Bugs</h1>

In [516]:
#prep up the bugs dataframe

bugs_list = []
for bug in bugs['issues']:
    bug['fields']['key'] = bug['key']
    for issuelink in bug['fields']['issuelinks']:
        try:
            if ((issuelink['outwardIssue']['fields']['issuetype']['name'] == 'Story') and 
            ((issuelink['type']['outward'] == 'associated with') or 
             (issuelink['type']['outward'] == 'relates to'))):
                bug['fields']['linkKey'] = issuelink['outwardIssue']['key']
                bug['fields']['linktype'] = issuelink['type']['outward']
        except:
            #print(issuelink)
            if ((issuelink['inwardIssue']['fields']['issuetype']['name'] == 'Story') and 
            ((issuelink['type']['inward'] == 'associated with') or 
             (issuelink['type']['inward'] == 'relates to'))):
                bug['fields']['linkKey'] = issuelink['inwardIssue']['key']
                bug['fields']['linktype'] = issuelink['type']['inward']
    #add each bug to bug list after updating the fields
    bugs_list.append(bug['fields'])
        
bugs_df = pd.DataFrame(bugs_list)

for column in bugs_df.columns:
    if ('custom' in column):
        bugs_df.rename(columns={column: idMap[column]}, inplace=True)
        
#bugs_df['Team'] = bugs_df['Team'].dropna().apply(lambda x: x[0].get('value') if (type(x) == list) else None)
#bugs_df['status'] = bugs_df['status'].dropna().apply(lambda x: x.get('name'))
#bugs_df['reporter'] = bugs_df['reporter'].dropna().apply(lambda x: x.get('name'))
bugs_df['fixVersions'] = bugs_df['fixVersions'].apply(lambda x: x[0]['name'] if ((type(x) == list) and x and (type(x[0]) == dict)) else None)
#bugs_df['Platform'] = bugs_df['Platform'].dropna().apply(lambda x: x[0].get('value'))
bugs_df['resolution'] = bugs_df['resolution'].apply(lambda x: x['name'] if type(x) == dict else None)
        
bugs_df['created'] = pd.to_datetime(bugs_df['created'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')

#insert a column for jira link
bugs_df['bug_link'] = '=HYPERLINK("' + domain + '/browse/' + bugs_df['key'] + '","' + bugs_df['key'] + '")'


In [421]:
%%html
<h1> Merge Stories, Epics and Sprints</h1>

In [422]:
#first merge - create the epics and stories merge
scope_df = pd.merge(epics_df, stories_df, how='right', on=None, left_on='key', right_on='Epic Link',
         left_index=False, right_index=False, sort=True,
         suffixes=('_epic', '_story'), copy=True, indicator=False,
         validate=None)

#Combine the sprints with the epics + stories dataframe and we can then drop the duplicate issue_key field.

sprintsWithStoriesAndEpics_df = pd.merge(scope_df, sprints_df, how='left', on=None, left_on='key_story', right_on='issue_key',
         left_index=False, right_index=False, 
         suffixes=('_story', '_sprint'),
         copy=True, indicator=False,
         validate=None).drop(columns = ['issue_key'])

In [423]:
%%html
<h1> Filter the stories, epics and sprints table now with the Quarter filter</h1>

In [424]:
#All stories from now on in this notebook will be between the R17 date rate: 04/03 - 07/05. The stories will have 
#dupicates because each story is on multiple rows (for each sprint the story is on)

sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['endDate'] > datetime(2018, 4, 3)]
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['endDate'] < datetime(2018, 7, 5)]

In [425]:
%%html
<h1> No of Stories and Epics</h1>

In [426]:
#Lets do some basic statistics
#get the number of unique stories - note that these stories are duplicated because they are part of multiple sprints
#in some cases.
# also this is the stories that were worked on and not necessariy finished. They were simply inside the sprints

print('No. Stories: ', sprintsWithStoriesAndEpics_df['key_story'].unique().size)
print('No. Epics: ', sprintsWithStoriesAndEpics_df['key_epic'].unique().size)

No. Stories:  413
No. Epics:  44


In [427]:
%%html
<h1> Check how many stories are not closed </h1>

In [428]:
#Check the stories which are not closed yet to see if we need to do anything special.
sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['status_story'] != 'Closed']['key_story'].unique().shape

(0,)

In [429]:
%%html
<h1> Average and Standard Deviation for Velocities for each team </h1>

In [430]:
#calculate the sprint velocities per team individually. We will use it to determine the standard deviation
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'Story Points', 'name']].copy()
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story', 'name']).agg({'Story Points':['sum']})

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.reset_index().drop(columns=['name'])
sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg( { 'mean', 'std'})

/Users/mbhatia/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Story Points            
                                  sum            
                                  std        mean
Team_story                                       
Admin Console Builder             NaN   42.000000
Admin Console Management          NaN    2.000000
Cache Back                  40.811996  105.428571
Core                              NaN   27.000000
End User Dashboard                NaN  118.000000
End User Responses                NaN   93.000000
Healers                     79.191224  115.666667
Karma                       32.000000   65.000000
Knight Riders               54.321829   71.111111
Mission Control              5.656854   12.000000
Optimus                     21.076956   32.714286
Seal Team                  121.498677  196.750000
Slide Tackle                 5.921711    9.666667
The Finer Tings Club        35.062190   52.250000

In [431]:
#Alternate velocity calculation that yields the same results as abov but no std. Commented off for now.
#Calculate the number of stories each team worked on, number of points each team covered, number of bugs
#each team fixed, number of features that were worked on.

#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'key_story', 'Story Points', 'name']].copy()
#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg({'key_story':['count'], 'Story Points':['sum'], 'name':['nunique']})
#sprintsWithStoriesAndEpics_dfCopy['average velocity'] = sprintsWithStoriesAndEpics_dfCopy['Story Points']['sum']/sprintsWithStoriesAndEpics_dfCopy['name']['nunique']

In [475]:
%%html
<h1> Calculate Story Spillover counts for all teams and the weighted average number of sprints required by team. Larger Velocity Standard Deviation should correlate with this. </h1>

In [496]:
#Calculate the spillover stories per team. How is spillover calculated? The sprint tags on a story are counted and if
#these are higher than one, then the story is categorized as spillover.

#first add up the number of sprints a story is in
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'key_story', 'name', 'startDate', 'endDate', 'Open Set To Date', 'Approval Set To Date', 'Closed Set To Date']].copy()

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['startDate'] - sprintsWithStoriesAndEpics_dfCopy['Open Set To Date']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] > -2


#This part is optional - detect stories that start approval before end of sprint and do not finish approval until after sprint.
sprintsWithStoriesAndEpics_dfCopy['Approval Lead Time'] = (sprintsWithStoriesAndEpics_dfCopy['endDate'] - sprintsWithStoriesAndEpics_dfCopy['Approval Set To Date']).dt.days
sprintsWithStoriesAndEpics_dfCopy['Close Delay'] = (sprintsWithStoriesAndEpics_dfCopy['Closed Set To Date'] - sprintsWithStoriesAndEpics_dfCopy['endDate']).dt.days
sprintsWithStoriesAndEpics_dfCopy['Approval Spill Over'] = (sprintsWithStoriesAndEpics_dfCopy['Approval Lead Time'] >= 1) & (sprintsWithStoriesAndEpics_dfCopy['Close Delay'] >= 1)

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy.duplicated(keep=False, subset='key_story')].to_excel(writer, index=False, sheet_name='Spillover Stories', freeze_panes=(1,0), columns=['Team_story', 'key_story', 'name', 'startDate', 'Open Set To Date', 'sprintLeadTime', 'sprintCommitment'])


In [497]:
#Comment the line below if we want to include all stories in sprint regardless of whether they were opened after sprint started.
#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['sprintCommitment']]

#Comment the line below if we want to include the Approval Spillovers.
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Approval Spill Over'] == False]


In [498]:
sprintsWithStoriesAndEpics_dfCopy= sprintsWithStoriesAndEpics_dfCopy.drop(columns = ['startDate', 'endDate', 'Open Set To Date', 'sprintCommitment', 'sprintLeadTime', 'Approval Set To Date', 'Closed Set To Date', 'Approval Lead Time', 'Close Delay', 'Approval Spill Over'])

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story', 'key_story']).agg({'name':['count']})

#reset index since we need to do another groupby
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.reset_index()

sprintsWithStoriesAndEpics_dfCopy['spillover sprint count'] = sprintsWithStoriesAndEpics_dfCopy['name']['count']
sprintsWithStoriesAndEpics_dfCopy= sprintsWithStoriesAndEpics_dfCopy.drop(columns = ['name'])
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story', 'spillover sprint count']).agg(['count'])
sprintsWithStoriesAndEpics_dfCopy.groupby(level=0).apply(max)
sprintsWithStoriesAndEpics_dfCopy


/Users/mbhatia/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


key_story
                                                         
                                                    count
Team_story               spillover sprint count          
Admin Console Builder    1                              3
Admin Console Management 1                              1
Cache Back               1                             17
                         2                             10
                         3                             15
                         4                              4
Core                     1                              7
End User Dashboard       1                             20
End User Responses       1                             11
Healers                  1                             17
                         2                             33
                         3                             10
                         4                              5
                         5                              1
                         7                              1
Karma                    1                              8
                         2                              6
                         3                              4
                         4                              2
                         5                              1
                         6                              2
Knight Riders            1                             26
                         2                              7
                         3                              4
                         4                              4
                         5                              6
Mission Control          1                              3
Optimus                  1                             13
                         2                             11
                         3                              1
Seal Team                1                             45
                         2                             29
                         3                              6
                         4                              4
                         5                              1
Slide Tackle             1                             12
                         2                              6
The Finer Tings Club     1                             30
                         2                             11
                         3                              2
                         4                              3

In [499]:
%%html
<h1> Calculate weighted average for no of sprints to complete a Story for each team </h1>

In [500]:
#reset index since we need to do another groupby
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.reset_index()

sprintsWithStoriesAndEpics_dfCopy['story count'] = sprintsWithStoriesAndEpics_dfCopy['key_story']['count']
sprintsWithStoriesAndEpics_dfCopy= sprintsWithStoriesAndEpics_dfCopy.drop(columns = ['key_story'])

#lets calculate the weighted average
sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).apply(lambda g: np.average(g['spillover sprint count'], weights=g['story count']))

/Users/mbhatia/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Team_story
Admin Console Builder       1.000000
Admin Console Management    1.000000
Cache Back                  2.130435
Core                        1.000000
End User Dashboard          1.000000
End User Responses          1.000000
Healers                     2.164179
Karma                       2.478261
Knight Riders               2.085106
Mission Control             1.000000
Optimus                     1.520000
Seal Team                   1.670588
Slide Tackle                1.333333
The Finer Tings Club        1.521739
dtype: float64

In [ ]:
#Repeat the above calculations by excluding stories that are spilling over due to approvals. If the story has
#an approval date that is before end of sprint and the close date is after then that story is considered an 
#approval spillover. Note that a story can be approval spillover for multiple sprints when it satisfies this.
#Engineering team wanted to see what this looks like. We expect an improvement in the numbers above. While this
#calculation should be meaningless, if the difference is dramatic, then it may be worth the exercise.

In [438]:
%%html
<h1> Calculate the total non-sprint bugs to total bugs ratio for the entire quarter </h1>

In [439]:
#ratio of bugs that were not linked to any stories to total bugs. Note that this is for the quarter for
#several releases in the quarter.

non_sprint_bugs_df = bugs_df[bugs_df['linkKey'].isnull()]
non_sprint_bugs_df = non_sprint_bugs_df[non_sprint_bugs_df['created'] > datetime(2018, 4, 3)]
non_sprint_bugs_df = non_sprint_bugs_df[non_sprint_bugs_df['created'] < datetime(2018, 7, 5)]
non_sprint_bugs_df['key'].unique().size/bugs_df['key'].unique().size

0.7770859277708593

In [440]:
#print the denominator - total number of bugs
bugs_df['key'].unique().size

803

In [441]:
%%html
<h1> Calculate the bugs created after code freeze with fixversion=R17 to total bugs created in the quarter marked with fixversion R17 </h1>

In [442]:
#Another way to analyze this is to find the bugs which were created after the last date of 
#the release 17 sprints that need to be fixed in the R17 release and compare with the total 
#number of bugs that were part of R17 (sprints and outside sprints). June 12 was the last date for R17. 

bugs_df1 = bugs_df[bugs_df['created'] > datetime(2018, 6, 17)]
#bugs_df1 = bugs_df1[bugs_df1['affectsVersions'] == 'R17']
bugs_df2 = bugs_df1[bugs_df1['fixVersions'] == 'R17']
n = bugs_df1['key'].unique().size

#find the bugs in the quarter that did not have a fixed version or had a fixed version of R17 or Triage
bugs_df1 = bugs_df[bugs_df['created'] > datetime(2018, 4, 3)]
bugs_df2 = bugs_df1[bugs_df1['fixVersions'] == 'R17']
bugs_df3 = bugs_df1[bugs_df1['fixVersions'] == 'Triage']
bugs_df4 = bugs_df1[bugs_df1['fixVersions'] == None]

total_bugs = bugs_df2['key'].unique().size + bugs_df3['key'].unique().size + bugs_df4['key'].unique().size

#ratio of bugs that were created after the main sprints were over that were marked to be fixed in R17 to
#bugs that were created in the whole quarter that are in Triage presently or without fixed version or 
#the fix version of R17
n/total_bugs


0.35119047619047616

In [447]:
%%html
<h1> Calculate the no. of bugs per story point </h1>

In [531]:
#combine the bugs with the stories dataframe
#find the number of bugs for each story point


storiesWithBugs_df = pd.merge(bugs_df, stories_df, how='right', on=None, left_on='linkKey', right_on='key',
         left_index=False, right_index=False, sort=True,
         suffixes=('_bug', '_story'), copy=True, indicator=False,
         validate=None)

#Comment this line below if we want to include bugs that were closed as invalid in some way. Does not appear to make 
#much difference.
storiesWithBugs_df = storiesWithBugs_df[~ storiesWithBugs_df['resolution'].isin(["Cannot Reproduce", "Declined", "Duplicate", "Invalid", "Not Needed", "Won't Do", "Won't Fix"])]

storiesWithBugs_df = storiesWithBugs_df[['Team_story', 'key_story', 'key_bug', 'Story Points_story']].copy().dropna()

storiesWithBugs_df = storiesWithBugs_df.groupby(['Team_story', 'key_story', 'Story Points_story']).agg(['count'])
storiesWithBugs_df = storiesWithBugs_df.reset_index()
storiesWithBugs_df['bugs per story point'] = storiesWithBugs_df['key_bug']['count']/storiesWithBugs_df['Story Points_story']

#write out the source data onto disk
storiesWithBugs_df.to_excel(writer, index=True, sheet_name='Bugs per Story Point', freeze_panes=(1,0))

storiesWithBugs_df = storiesWithBugs_df.drop(columns = ['Story Points_story', 'key_story', 'key_bug'])
storiesWithBugs_df = storiesWithBugs_df.groupby(['Team_story']).agg(['mean'])
storiesWithBugs_df = storiesWithBugs_df.reset_index()


storiesWithBugs_df['avg bugs per story point'] = storiesWithBugs_df['bugs per story point']['mean']
storiesWithBugs_df= storiesWithBugs_df.drop(columns = ['bugs per story point'])

storiesWithBugs_df = storiesWithBugs_df.sort_values(by='avg bugs per story point', ascending=False)
storiesWithBugs_df

/Users/mbhatia/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,Team_story,avg bugs per story point
,,
,,
3,Knight Riders,0.387919
7,The Finer Tings Club,0.294274
2,Karma,0.278312
1,Healers,0.266667
0,Cache Back,0.210363
6,Seal Team,0.183384
4,Mission Control,0.125000
5,Optimus,0.115385


In [465]:
%%html
<h1> Analyze  Stories opened after sprint start date and calculate the weighted average of this number per team. Repeat this for regression period.</h1>

In [468]:
#find the stories which were inserted in sprints after sprints started

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'startDate', 'Open Set To Date', 'reporter_story', 'Story Points', 'key_story', 'name', 'fixVersions_story']].copy()

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['startDate'] - sprintsWithStoriesAndEpics_dfCopy['Open Set To Date']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] > -2
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] != True].sort_values(by='key_story')
#sprintsWithStoriesAndEpics_dfCopy['key_story'].unique().size

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
sprintsWithStoriesAndEpics_dfCopy.to_excel(writer, index=False, sheet_name='Late Commitments to Sprint', freeze_panes=(1,0), columns=['Team_story', 'startDate', 'Open Set To Date', 'reporter_story', 'Story Points', 'key_story', 'name', 'sprintLeadTime', 'sprintCommitment'])

In [470]:
#Lets calculate for R17 how many stories were inserted after the developer sprints were over. This is a subset of 
#the previous set.

storiesPostLastSprint_df = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['startDate'] > datetime(2018, 6, 5)]
storiesPostLastSprint_df = storiesPostLastSprint_df[storiesPostLastSprint_df['fixVersions_story'] == 'R17']


In [472]:
#drop unnecessary columns before we do stats
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.drop(columns=['startDate', 'Open Set To Date', 'Story Points', 'name', 'sprintCommitment', 'reporter_story'])
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg({'sprintLeadTime':['mean'], 'key_story':['count']})

#we must filter the noise
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['key_story']['count'] > 10]
sprintsWithStoriesAndEpics_dfCopy

,sprintLeadTime,key_story
,mean,count
Team_story,,
Cache Back,-5.947368,19
Healers,-7.424242,33
Knight Riders,-12.517241,29
Optimus,-6.428571,15
Seal Team,-7.950000,60
The Finer Tings Club,-5.076923,14


In [501]:
%%html
<h2> Stories opened post start of regression.</h2>

In [474]:
#drop unnecessary columns before we do stats
storiesPostLastSprint_dfCopy = storiesPostLastSprint_df.drop(columns=['startDate', 'Open Set To Date', 'Story Points', 'name', 'sprintCommitment', 'reporter_story'])
storiesPostLastSprint_dfCopy = storiesPostLastSprint_dfCopy.groupby(['Team_story']).agg({'sprintLeadTime':['mean'], 'key_story':['count']})

storiesPostLastSprint_dfCopy

,sprintLeadTime,key_story
,mean,count
Team_story,,
Cache Back,-7.50,2
Knight Riders,-6.75,4
Optimus,-2.00,1
Seal Team,-6.75,4
The Finer Tings Club,-3.00,1


In [319]:
%%html
<h1> Calculate the Stories spilled over in the Testing State and total time to test for them on average and lead time provided by developers on average.</h1>

In [502]:
#number of issues left in Testing and Testing lead time inside sprint
#find the issues that are still in Testing before the end of their sprint. Only include issues that were committed 
#to in the beginning of the sprint.

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'startDate', 'endDate', 'Testing Set To Date', 'Approval Set To Date', 'Approval Set By', 'Open Set To Date', 'key_story']].copy()
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.dropna()
sprintsWithStoriesAndEpics_dfCopy['Testing Lead Time'] = (sprintsWithStoriesAndEpics_dfCopy['endDate'] - sprintsWithStoriesAndEpics_dfCopy['Testing Set To Date']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Testing Lead Time'] >= 2] 
sprintsWithStoriesAndEpics_dfCopy['Ready for Approval Delay'] = (sprintsWithStoriesAndEpics_dfCopy['Approval Set To Date'] - sprintsWithStoriesAndEpics_dfCopy['endDate']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Ready for Approval Delay'] >= 2]

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['startDate'] - sprintsWithStoriesAndEpics_dfCopy['Open Set To Date']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] > -2

#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['sprintCommitment']].sort_values(by='key_story')
sprintsWithStoriesAndEpics_dfCopy['Testing Time'] = sprintsWithStoriesAndEpics_dfCopy['Testing Lead Time'] + sprintsWithStoriesAndEpics_dfCopy['Ready for Approval Delay']

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
sprintsWithStoriesAndEpics_dfCopy.to_excel(writer, index=False, sheet_name='Testing Spillovers in Sprints', freeze_panes=(1,0), columns=['Team_story', 'startDate', 'endDate', 'Testing Set To Date', 'Approval Set To Date', 'Approval Set By', 'Open Set To Date', 'key_story', 'Testing Lead Time', 'Ready for Approval Delay', 'Testing Time'])

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.sort_values(by='key_story').drop(columns = ['Ready for Approval Delay', 'startDate', 'endDate', 'Testing Set To Date', 'Approval Set To Date', 'Open Set To Date', 'sprintLeadTime', 'sprintCommitment', 'Approval Set By'])

sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg({'key_story':['count'], 'Testing Lead Time':['mean'], 'Testing Time':['mean']})

,key_story,Testing Lead Time,Testing Time
,count,mean,mean
Team_story,,,
Admin Console Builder,1,11.000000,13.000000
Cache Back,23,11.565217,22.347826
Healers,22,11.590909,22.454545
Karma,8,8.875000,20.750000
Knight Riders,14,15.785714,25.642857
Mission Control,2,10.000000,62.500000
Optimus,5,6.000000,14.000000
Seal Team,16,11.687500,23.437500


In [321]:
%%html
<h1> Calculate the Stories spilled over in the Approval State and total time to Approve on average and lead time provided by testers on average.</h1>

In [503]:
#number of issues left in Approval and Approval lead time inside sprint

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'startDate', 'endDate', 'Approval Set To Date', 'Approval Set By', 'Closed Set By', 'Open Set To Date', 'Closed Set To Date', 'key_story']].copy()
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.dropna()

sprintsWithStoriesAndEpics_dfCopy['Approval Lead Time'] = (sprintsWithStoriesAndEpics_dfCopy['endDate'] - sprintsWithStoriesAndEpics_dfCopy['Approval Set To Date']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Approval Lead Time'] >= 2] 
sprintsWithStoriesAndEpics_dfCopy['Close Delay'] = (sprintsWithStoriesAndEpics_dfCopy['Closed Set To Date'] - sprintsWithStoriesAndEpics_dfCopy['endDate']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Close Delay'] >= 2]

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['startDate'] - sprintsWithStoriesAndEpics_dfCopy['Open Set To Date']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] > -2
#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['sprintCommitment']].sort_values(by='key_story')
sprintsWithStoriesAndEpics_dfCopy['Approval Time'] = sprintsWithStoriesAndEpics_dfCopy['Approval Lead Time'] + sprintsWithStoriesAndEpics_dfCopy['Close Delay']

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
sprintsWithStoriesAndEpics_dfCopy.to_excel(writer, index=False, sheet_name='Approval Spillovers in Sprints', freeze_panes=(1,0), columns=['Closed Set By', 'Team_story', 'startDate', 'endDate', 'Approval Set To Date', 'Approval Set By', 'Closed Set By', 'Open Set To Date', 'Closed Set To Date', 'key_story', 'Approval Lead Time', 'Close Delay', 'Approval Time'])

#sprintsWithStoriesAndEpics_dfCopy
sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg({'key_story':['count'], 'Approval Lead Time':['mean'], 'Approval Time':['mean']})

,key_story,Approval Lead Time,Approval Time
,count,mean,mean
Team_story,,,
Admin Console Builder,1,25.000000,27.000000
Cache Back,5,10.400000,26.200000
Healers,30,14.666667,27.200000
Karma,4,3.750000,8.750000
Knight Riders,1,3.000000,5.000000
Optimus,1,6.000000,9.000000
Seal Team,26,12.923077,26.923077
The Finer Tings Club,2,9.500000,15.500000


In [323]:
%%html
<h1> Calculate the Stories spilled over in the Open/InProgress States and average development time on these stories.</h1>

In [505]:
#number of stories left in open or in progress state
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Open Set To Date', 'Team_story', 'startDate', 'endDate', 'In Progress Set To Date', 'In Progress Set By', 'Testing Set By', 'Testing Set To Date', 'key_story']].copy()
sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy.dropna()

sprintsWithStoriesAndEpics_dfCopy['Dev Lead Time'] = (sprintsWithStoriesAndEpics_dfCopy['endDate'] - sprintsWithStoriesAndEpics_dfCopy['In Progress Set To Date']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Dev Lead Time'] >= 2] 
sprintsWithStoriesAndEpics_dfCopy['Testing Delay'] = (sprintsWithStoriesAndEpics_dfCopy['Testing Set To Date'] - sprintsWithStoriesAndEpics_dfCopy['endDate']).dt.days

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['Testing Delay'] >= 2]

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['startDate'] - sprintsWithStoriesAndEpics_dfCopy['Open Set To Date']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] > -2

#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_dfCopy[sprintsWithStoriesAndEpics_dfCopy['sprintCommitment']].sort_values(by='key_story')
sprintsWithStoriesAndEpics_dfCopy['Dev Time'] = sprintsWithStoriesAndEpics_dfCopy['Dev Lead Time'] + sprintsWithStoriesAndEpics_dfCopy['Testing Delay']

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
sprintsWithStoriesAndEpics_dfCopy.to_excel(writer, index=False, sheet_name='In Progress Sprint Spillovers', freeze_panes=(1,0), columns=['Open Set To Date', 'In Progress Set By', 'Team_story', 'startDate', 'endDate', 'In Progress To Date', 'In Progress Set By', 'key_story', 'Dev Lead Time', 'Testing Delay', 'Dev Time'])

sprintsWithStoriesAndEpics_dfCopy.groupby(['Team_story']).agg({'key_story':['count'], 'Dev Lead Time':['mean'], 'Dev Time':['mean']})


/Users/mbhatia/anaconda3/lib/python3.6/site-packages/pandas/io/formats/excel.py:372: FutureWarning: Not all names specified in 'columns' are found; this will raise a KeyError in the future
  FutureWarning)


,key_story,Dev Lead Time,Dev Time
,count,mean,mean
Team_story,,,
Cache Back,8,12.000000,28.125000
Healers,17,19.470588,32.411765
Karma,16,16.125000,34.250000
Knight Riders,15,15.800000,30.000000
Mission Control,1,5.000000,44.000000
Optimus,2,3.500000,5.500000
Seal Team,4,7.500000,15.500000
Slide Tackle,1,3.000000,5.000000


In [325]:
%%html
<h1> Calculate the Stories not having any mention of AC or Acceptance.</h1>

In [326]:
#this is a list of strings
#scope_df['textinfo'] = scope_df['textinfo_story'] + scope_df['textinfo_epic']
scope_df['textinfo'] = scope_df['textinfo_story']

In [327]:
scope_df['Invalid AC'] = scope_df['textinfo'].str.contains('Acceptance|AC', case = False, regex = True) == False

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
scope_df[scope_df['Invalid AC']].to_excel(writer, index=False, sheet_name='Invalid AC', freeze_panes=(1,0), columns=['Team_story', 'key_story', 'reporter_story'])


In [328]:
invalid_ac_df = scope_df[['reporter_story', 'Invalid AC']].copy()

In [329]:
#produce statistics for valid/invalid AC
invalid_ac_df.groupby(['reporter_story']).sum().sort_values(by=['Invalid AC'], ascending=False).head()

,Invalid AC
reporter_story,
sami.ahmed,7.0
kyle.mills,4.0
jayanth.prathipati,4.0
addisu.alemu,4.0
vishal.bhosale,1.0


In [ ]:
writer.save()

In [ ]:
%%html
<h1> Phase 2: Learning.</h1>

In [ ]:
dataset1 = pd.DataFrame(scope_df, columns = ['Epic Name', 'textinfo'])
dataset1.dropna(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#create document vectors

vectorizer = TfidfVectorizer()
vectors_ds1 = vectorizer.fit_transform(dataset1.textinfo)

In [ ]:
#split this into training and test data
from sklearn.model_selection import train_test_split

predictors = vectors_ds1
targets = dataset1['Epic Name']

pred_train, pred_test, tar_train, tar_test  =   train_test_split(predictors, targets, test_size=.20)

In [ ]:
clf = MultinomialNB()
clf.fit(pred_train, tar_train)

In [ ]:
predictions = clf.predict(pred_test)

In [ ]:
import sklearn.metrics

sklearn.metrics.confusion_matrix(tar_test,predictions)
sklearn.metrics.accuracy_score(tar_test, predictions)

In [ ]:
pred_train.size

In [ ]:
vectors_ds1